In [49]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()
pd.set_option('display.max_rows', 5000)


In [85]:
zridf=pd.read_csv('../data/zri_multifamily_v2.csv')
zridf=zridf[['zip','City','State','Metro','CountyName']].drop_duplicates()

In [19]:
trendslist=pd.read_csv('../data/googletrends_countries.csv')
trendslist=trendslist[trendslist['country_code']=='US']

In [105]:
name_dict={
    'MAprovidence-warwick':'MAmassachusetts',
    'NHboston-cambridge-newton':'NHnew hampshire',
    'MEnan':'MEmaine',
    'NJnew york-newark-jersey city':'NJnew jersey',
    'INchicago-naperville-elgin':'INindiana',
    'WIchicago-naperville-elgin':'WIwisconsin',
    'MOst. louis':'MOmissouri',
    'HIurban honolulu':'HIhawaii',
}

In [24]:
trendslist2

,Unnamed: 0,country_code,sub_code,name
3794,3795,US,NaN,united states
3795,3796,US,US-AL,alabama
3796,3797,US,US-AK,alaska
3797,3798,US,US-AZ,arizona
3798,3799,US,US-AR,arkansas
...,...,...,...,...
117199,126046,US,US-658,green bay-appleton wi
117200,126048,US,US-669,madison wi
117201,126049,US,US-617,milwaukee wi
117202,126051,US,US-705,wausau-rhinelander wi


In [78]:
trendslist2=trendslist.copy()
trendslist2['name']=trendslist2['name'].map(lambda x: str(x).lower())
trendslist2['state']=trendslist2['sub_code'].apply(lambda x: str(x)[-2:] if str(x)[:3]=='US-' else '0')
trendslist2['state_name']=trendslist2['state']+trendslist2['name']
#trendslist2['state_name_simplified']=trendslist2['state_name'].map(lambda x: x[:12])
trendslist2['state_name_simplified']=trendslist2['state_name'].map(lambda x: x[:x.find('-')] if x.find('-')!=-1 else x)


In [115]:
zridf['CountyName']=zridf['CountyName'].map(lambda x: str(x).lower())
zridf['Metro']=zridf['Metro'].map(lambda x: str(x).lower())
zridf['Metro_State']=zridf['State']+zridf['Metro']
#zridf['Metro_State_simplified']=zridf['Metro_State'].map(lambda x: x[:12])
zridf['Metro_State_simplified']=zridf['Metro_State'].map(lambda x: name_dict[x] if x in name_dict.keys() else x)
zridf['Metro_State_simplified']=zridf['Metro_State_simplified'].map(lambda x: x[:x.find('-')] if x.find('-')!=-1 else x)


In [116]:
#merged=zridf.merge(trendslist2, how='left', left_on='Metro_State', right_on='state_name')
#merged=merged.merge(trendslist2, how='left', left_on='Metro_State_simplified', right_on='state_name_simplified')
merged=zridf.merge(trendslist2, how='left', left_on='Metro_State_simplified', right_on='state_name_simplified')


In [120]:
merged['sub_code'].unique()

array(['US-MA', 'US-RI', 'US-NH', 'US-ME', 'US-CT', 'US-NJ', 'US-NY',
       'US-PA', 'US-DC', 'US-VA', 'US-MD', 'US-WV', 'US-NC', 'US-SC',
       'US-GA', 'US-FL', 'US-TN', 'US-KY', 'US-OH', 'US-IN', 'US-MI',
       'US-IA', 'US-WI', 'US-MN', 'US-ND', 'US-MT', 'US-IL', 'US-MO',
       'US-KS', 'US-AR', 'US-OK', 'US-TX', 'US-CO', 'US-UT', 'US-AZ',
       'US-NM', 'US-NV', 'US-CA', 'US-HI', 'US-OR', 'US-WA', 'US-AK'],
      dtype=object)

In [146]:
state_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [147]:
state_list_ext=[]

for state in state_list:
    state_list_ext.append("US-" + state)

In [159]:
kw_list=['Fire','Water damage','Smoke','Poison','Flooding','storm','hurricane','tornado','forest fire']

df2=pd.DataFrame(columns=['date','isPartial','State'])
df2=df2.set_index('date')

for state in state_list_ext[0:2]:
    pytrend.build_payload(kw_list, cat=None, timeframe='2010-01-01 2021-01-01', geo=state) 

    df = pytrend.interest_over_time(backoff_factor=60, sleep=60)
    df['State']=state
    df2=df2.append(df)
    

ResponseError: The request failed: Google returned a response with code 400.

In [149]:
df2

,isPartial,State
date,,


In [157]:
kw_list=['Taylor Swift']
pytrend.build_payload(kw_list, cat=None, timeframe='2014-01-01 2021-01-01', geo='US-AL') 

# get the interest over time
df = pytrend.interest_over_time()
df

,Taylor Swift,isPartial
date,,
2014-01-01,33,False
2014-02-01,29,False
2014-03-01,18,False
2014-04-01,28,False
2014-05-01,20,False
2014-06-01,21,False
2014-07-01,18,False
2014-08-01,49,False
2014-09-01,37,False
